## Subgraphs
- Shared schema keys
- 

In [3]:
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, END, add_messages, START
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import AIMessage, HumanMessage
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode

In [ ]:
load_dotenv()

class SubState(TypedDict):
    messages: Annotated[list, add_messages]
# Substate as child state
search_tool = TavilySearchResults(max_results=2)
tools = [search_tool]

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
llm_with_tools = llm.bind_tools(tools)

In [5]:
def agent(state: SubState):
    return {
        "messages": [llm_with_tools.invoke(state["messages"])]
    }

def router(state: SubState):
    prev_message = state["messages"][-1]

    if(hasattr(prev_message,"tool_calls") and len(prev_message.tool_calls) > 0):
        return "tool"
    else:
        return "end"

In [15]:
tool_node = ToolNode(tools)

subgraph = StateGraph(SubState)

subgraph.add_node("agent", agent)
subgraph.add_node("tool_node", tool_node)

subgraph.add_edge(START, "agent")
subgraph.add_conditional_edges("agent", router, {"tool":"tool_node", "end": END})
subgraph.add_edge("tool_node", "agent")

child_app = subgraph.compile()

In [16]:
child_app.invoke({"messages": [HumanMessage(content="How is the weather in Vaasa, Fimland today")]})

{'messages': [HumanMessage(content='How is the weather in Vaasa, Fimland today', additional_kwargs={}, response_metadata={}, id='56ee18f7-417e-4888-9dc5-b8c0568a1968'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in Vaasa, Finland today"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--c19012ae-a6a5-4839-99d7-f2a617ff4ce8-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Vaasa, Finland today'}, 'id': 'bad4a266-a066-452c-aba4-4315f5a88b59', 'type': 'tool_call'}], usage_metadata={'input_tokens': 60, 'output_tokens': 16, 'total_tokens': 76, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content='[{"title": "Vaasa weather in June 2025 - Weather25.com", "url": "https://www.weather25.com/europe/finland/western-finland/vaasa?page=month

### 1) Multi-agent with parent and child graph
- Shared schema keys

In [29]:
from typing import List


class ParentState(TypedDict):
    messages: Annotated[List, add_messages]

parent_graph = StateGraph(ParentState)

# Adding compiled child_graph-child_app as a node to parent_graph
parent_graph.add_node("search_agent", child_app)

parent_graph.add_edge(START, "search_agent")
parent_graph.add_edge("search_agent", END)

parent_app = parent_graph.compile()

outcome = parent_app.invoke({"messages": [HumanMessage(content="How is the weather in Vaasa, Finland today")]})
# outcome = parent_app.stream({"messages": [HumanMessage(content="How is the weather in Vaasa, Finland today")]}, stream_mode="values")
outcome


{'messages': [HumanMessage(content='How is the weather in Vaasa, Finland today', additional_kwargs={}, response_metadata={}, id='4bce055b-e551-4376-8c1d-36f8f30ba1c2'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in Vaasa, Finland today"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--640bdfa7-b070-49a5-ab2a-c8659d9ea938-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Vaasa, Finland today'}, 'id': '8f5bb93a-d01c-42f6-ba6d-5b83a32e5c10', 'type': 'tool_call'}], usage_metadata={'input_tokens': 59, 'output_tokens': 16, 'total_tokens': 75, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content='[{"title": "Vaasa January 2025 Historical Weather Data (Finland)", "url": "https://weatherspark.com/h/m/87765/2025/1/Historical-Weather-in

In [31]:
# for chunck in outcome:
#     print(chunck["messages"][-1].pretty_print())